In [49]:
# create a model that uses the fenchel-young loss and trains on 200 images 
# the images are permuted in 9 tiles and in every iteration a new permutation is generated.
# we use a deep encoder with fixed parameters, adam optimizer, preprocessing before tiling  

import tensorflow as tf

In [50]:
from ast import Pass
import platform
import cv2
import numpy as np
import os
from pathlib import Path

number_of_images = 250

root = os.getcwd() # Don't change this
data_dirname = '/data_test/plantvillage/' # Change as you like 
p = Path(root + data_dirname)
p.mkdir(exist_ok=True) 
if platform.system()=='Darwin':
  root = os.getcwd() # Don't change this
  data_dirname = '/data_test/plantvillage/' # Change as you like 
  p = Path(root + data_dirname)
  p.mkdir(exist_ok=True) 
else:
  #p = Path("C:/Users/mwels/Documents/Uni/11. Semester/Deep learning in visual recognition/Plant_leave_diseases_dataset_without_augmentation")
  #p.mkdir(exist_ok=True)
  pass



classes = [
  'Apple___Apple_scab',
  'Apple___healthy',
  'Apple___Black_rot',
  'Apple___Cedar_apple_rust',
  "all"
  ]

if "all" in classes:
  classes = os.listdir(p)

for c in classes:
  print(c,end=" ")
  filelist = [x for x in (p/c).iterdir() if x.is_file()]
  for f in filelist:
    img = cv2.imread(str(f))
    if img is None:
      print(f'Failed to open {f}. Deleting file')
      os.remove(str(f))


filelist
print(len(filelist))

Apple___Apple_scab Apple___Black_rot Apple___Cedar_apple_rust Apple___healthy 1645


In [51]:
len(filelist)

1645

In [52]:
tilex = 2
number_of_tiles = tilex**2
number_of_permutations = 24
tile_size = 255//tilex
sfmax = True

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

conv_base = MobileNet(weights='imagenet',
                      include_top=False,
                      input_shape=(tile_size, tile_size, 3))

In [53]:
from keras.layers import Dense, Concatenate, Input, Flatten, Lambda 
from keras.models import Model
import os


tiles = Input((number_of_tiles,tile_size,tile_size,3))
inputs = {}
layers = {}
embedds = {}

shared_conv = conv_base 

for i in range(number_of_tiles):
    inputs[f'tiles{i}'] = Input((tile_size,tile_size,3))
    #layers[f'tile{i}'] = Lambda(lambda x: x[:,i,:,:,:])(tiles)

    layers[f'deep_layers{i}'] = shared_conv(inputs[f'tiles{i}'])
    #layers[f'deep_layers{i}'] = shared_conv(layers[f'tile{i}'])
    embedds[f'embedd{i}'] = Flatten()(layers[f'deep_layers{i}'])

concatonation = Concatenate(axis=1)(list(embedds.values()))

out = Dense(number_of_tiles*10, activation="relu", kernel_initializer='he_normal')(concatonation)
if sfmax:
    out = Dense(number_of_permutations, activation="softmax", kernel_initializer='he_normal')(out)
else:
    out = Dense(number_of_tiles, kernel_initializer='he_normal')(out)
out = Flatten()(out)

#model = Model(inputs=tiles, outputs=out)
model = Model(inputs=list(inputs.values()), outputs=out)

In [54]:
tf.keras.utils.plot_model(model, 
    show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [55]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 127, 127, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_23 (InputLayer)          [(None, 127, 127, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_24 (InputLayer)          [(None, 127, 127, 3  0           []                               
                                )]                                                          

In [56]:
total_num_layers = len(model.layers)
num_base_layers = len(conv_base.layers)
print(f"Total number of layers is {total_num_layers}")
print(f"Number of pretrained base layers is {num_base_layers}")

for layer in model.layers[:num_base_layers]:
    layer.trainable=True
for layer in model.layers[num_base_layers:]:
    layer.trainable=True

Total number of layers is 13
Number of pretrained base layers is 86


In [57]:
sys.path.append('src')

In [58]:
from keras import optimizers
from sklearn.model_selection import train_test_split
from src.PermOneHotDataGen import *
from src.model_tools import *
from src.permutation_tools import *

x_train, x_test = train_test_split(filelist)

train_generator = PermOneHotDataGen(x_train,
                                    batch_size=8,
                                    tilenumberx=tilex, 
                                    shuffle_permutations=True)

validation_generator = PermOneHotDataGen(x_test,
                                        batch_size=8,
                                        tilenumberx=tilex,
                                        shuffle_permutations=True)

train_generator_sm = PermOneHotDataGen(x_train,
                                    batch_size=256,
                                    tilenumberx=tilex, max_perms=number_of_permutations)

validation_generator_sm = PermOneHotDataGen(x_test,
                                        batch_size=256,
                                        tilenumberx=tilex, max_perms=number_of_permutations)


optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name='Adam',
)

if sfmax:
    model.compile(optimizer=optimizer,
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
else:
    model.compile(
        optimizer=optimizer,
        loss=RankingLoss(),
        metrics=[ProjectedRanksAccuracy(), PartialRanksAccuracy()])


In [59]:
print(train_generator.next()[1].shape)
print(train_generator_sm.next()[1].shape)
print(validation_generator.next()[1].shape)
print(validation_generator_sm.next()[1].shape)
print(train_generator.next()[0].shape)
print(train_generator_sm.next()[0].shape)
print(validation_generator.next()[0].shape)
print(validation_generator_sm.next()[0].shape)

(8, 4)
(256, 24)
(8, 4)
(256, 24)
(4, 8, 127, 127, 3)
(4, 256, 127, 127, 3)
(4, 8, 127, 127, 3)
(4, 156, 127, 127, 3)


In [60]:
ProjectedRanksAccuracy().update_state(validation_generator.next()[1], validation_generator.next()[1])

In [61]:
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output

class PlotLearning(tf.keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        for metric in logs:
            self.metrics[metric] = []
            

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        f, axs = plt.subplots(1, len(metrics), figsize=(15,5))
        clear_output(wait=True)

        for i, metric in enumerate(metrics):
            axs[i].plot(range(1, epoch + 2), 
                        self.metrics[metric], 
                        label=metric)
            if logs['val_' + metric]:
                axs[i].plot(range(1, epoch + 2), 
                            self.metrics['val_' + metric], 
                            label='val_' + metric)
                
            axs[i].legend()
            axs[i].grid()

        plt.tight_layout()
        plt.show()

In [62]:
train_unpack_sm = PermOneHotDataGen(x_train,
                                    batch_size=len(x_train),
                                    tilenumberx=tilex, max_perms=number_of_permutations)

validation_unpack_sm = PermOneHotDataGen(x_test,
                                        batch_size=len(x_test),
                                        tilenumberx=tilex, max_perms=number_of_permutations)
#"loaded" data:
x_t, y_t = train_unpack_sm.next()
x_v, y_v = validation_unpack_sm.next()


In [63]:
x_l = [x_t[i] for i in range(number_of_tiles)]

In [64]:
nb_epochs = 30

model.fit(x=x_l,
          y=y_t,
          epochs = nb_epochs,
          verbose=1)

Epoch 1/30
39/39 [==============================] - 77s 2s/step - loss: 11.2677 - accuracy: 0.1971
Epoch 2/30
39/39 [==============================] - 72s 2s/step - loss: 10.6653 - accuracy: 0.3163
Epoch 3/30
39/39 [==============================] - 76s 2s/step - loss: 10.3432 - accuracy: 0.3374
Epoch 4/30
39/39 [==============================] - 76s 2s/step - loss: 9.9418 - accuracy: 0.3788
Epoch 5/30
39/39 [==============================] - 76s 2s/step - loss: 9.9454 - accuracy: 0.3771
Epoch 6/30
39/39 [==============================] - 77s 2s/step - loss: 9.9787 - accuracy: 0.3779
Epoch 7/30
27/39 [===================>..........] - ETA: 24s - loss: 10.1735 - accuracy: 0.3634

KeyboardInterrupt: 

In [ ]:
nb_epochs = 30

model.fit(train_generator_sm,
          epochs = nb_epochs,
          validation_data=validation_generator_sm,
          verbose=1)

Epoch 1/30
39/39 [==============================] - 16s 326ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 2/30
39/39 [==============================] - 12s 319ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 3/30
39/39 [==============================] - 13s 321ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 4/30
39/39 [==============================] - 13s 332ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 5/30
39/39 [==============================] - 13s 333ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 6/30
39/39 [==============================] - 13s 330ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 7/30
39/39 [==============================] - 13s 326ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 8/30
39/39 [==============================] - 12s 318ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 9/30
39/39 [==============================] - 13s 336ms/step - loss: 4.7898 - accuracy: 0.0381
Epoch 10/30
39/39 [==============================] - 13s 330ms/step - loss: 4.7898 - accura

KeyboardInterrupt: 

In [ ]:
#model.save("models/2022_10_27__01")